# An example of using the python eTaSL driver in a python notebook

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML
%matplotlib inline
%load_ext autoreload
%autoreload 2

## 1. eTaSL definition

We start by defining the KUKA LWR robot in eTaSL specification string.  Here a manual definition is chosen in order
to have a self-contained example; as an alternative, an URDF-file could be loaded. We define
a Lua/eTaSL function specifying the robot:

In [ ]:
etasl1 = """
require("context")
require("geometric")


function define_robot_LWR(ctx,name,base, vel)
    if name == nil then
        name = ""
    end
    if #name~=0 then
        name = name .. ":"
    end
    local L1 = constant(0.310)
    local L2 = constant(0.4)
    local L3 = constant(0.390)
    local L4 = constant(0.078)
    q1=Variable{context=ctx, name="q1", vartype="robot"}
    q2=Variable{context=ctx, name="q2", vartype="robot"}
    q3=Variable{context=ctx, name="q3", vartype="robot"}
    q4=Variable{context=ctx, name="q4", vartype="robot"}
    q5=Variable{context=ctx, name="q5", vartype="robot"}
    q6=Variable{context=ctx, name="q6", vartype="robot"}
    q7=Variable{context=ctx, name="q7", vartype="robot"}
    robot = {}
    robot.elbow = cached(base * rotate_z(q1) * translate_z(L1) * rotate_x(q2) * rotate_z(q3) * translate_z(L2) * rotate_x(q4))
    robot.wrist = cached(robot.elbow * rotate_z(q5) * translate_z(L3))
    robot.ee    = cached(robot.wrist * rotate_x(q6) * rotate_z(q7) * translate_z(L4))

    robot.q1=q1
    robot.q2=q2
    robot.q3=q3
    robot.q4=q4
    robot.q5=q5
    robot.q6=q6
    robot.q7=q7

    BoxConstraint{context=ctx, var_name="q1", lower=-vel , upper=vel }
    BoxConstraint{context=ctx, var_name="q2", lower=-vel , upper=vel }
    BoxConstraint{context=ctx, var_name="q3", lower=-vel , upper=vel }
    BoxConstraint{context=ctx, var_name="q4", lower=-vel , upper=vel }
    BoxConstraint{context=ctx, var_name="q5", lower=-vel , upper=vel }
    BoxConstraint{context=ctx, var_name="q6", lower=-vel , upper=vel }
    BoxConstraint{context=ctx, var_name="q7", lower=-vel , upper=vel }
    return robot
end
""";



The specification describes a robot that follows a given trajectory.  This trajectory is given
as an input (**tgt_x**, **tgt_y**, **tgt_z**) that comes 
from outside the controller.

To demonstrate the use of feature variables, a feature variable **f1** is introduced.  (This example is a bit artificial)

In [ ]:
etasl2 = """
tgt_x = ctx:createInputChannelScalar("tgt_x",0.7)
tgt_y = ctx:createInputChannelScalar("tgt_y",0)
tgt_z = ctx:createInputChannelScalar("tgt_z",0.7)

-- joint velocity limits:
--vel=0.1  -- try this
vel =  0.5
r = define_robot_LWR(ctx,"",frame(constant(Vector(0,0,0))), vel)

f1=Variable{context=ctx, name="f1", vartype="feature"}


Constraint{
    context=ctx,
    name="x",
    expr = tgt_x - coord_x(origin(robot.ee)),
    priority = 2,
    K        = 4
}

Constraint{
    context=ctx,
    name="y",
    expr = tgt_y - coord_y(origin(robot.ee)),
    priority = 2,
    K        = 4
}

Constraint{
    context=ctx,
    name="z",
    expr = tgt_z - coord_z(origin(robot.ee)),
    priority = 2,
    K        = 4
}

Constraint{
    context=ctx,
    name="f",
    expr = f1 - tgt_y,
    priority = 2,
    K        = 1,
    weight   = 0.01
}
ctx:setOutputExpression("ee_x",coord_x(origin(robot.ee)))
ctx:setOutputExpression("ee_y",coord_y(origin(robot.ee)))
ctx:setOutputExpression("ee_z",coord_z(origin(robot.ee)))

""";



In [ ]:
etaslspec=etasl1+etasl2

# Simulating the eTaSL controller using the low-level eTaSL driver

In [ ]:
T=20
dt=0.01
df = pd.DataFrame()
df['time'] = np.arange(0,T,dt)
df['tgt_x']= 0
df['tgt_y']= np.sin(df['time'])*0.15-0.7
df['tgt_z']= 0.3
# the full table in HTML: 
# HTML(df.style.set_precision(3).render())

# the first 5 lines:
df.head()

## 2. Simulating the eTaSL controller using the python eTaSL driver

### 2.1 Define the inputs to the controller

In [ ]:
# define the time period for simulation:
T    = 20
dt   = 0.01
time = np.arange(0,T,dt)

# define INPUT (and its time derivative)
INP_LBL = ['tgt_x','tgt_y','tgt_z']  # corresponds to eTaSL definition
INP      = np.zeros((len(time),3))
INP[:,0] = time*0
INP[:,1] = np.sin(time)*0.15 - 0.7
INP[:,2] = time*0 + 0.3

INPVEL   = np.zeros((len(time),3))
INPVEL[:,0] = time*0
INPVEL[:,1] = np.cos(time)*0.15
INPVEL[:,2] = time*0


### 2.2 define the state (both robot- and feature variables) and output arrays

These are the arrays that will contain the history of the state variables for both robot- and feature variables, and the output.

In [ ]:
# robot variables:
J_LBL = ['q1','q2','q3','q4','q5','q6','q7']
JPOS     = np.zeros((len(time),7))
JVEL     = np.zeros((len(time),7))

# feature variables:
F_LBL = ['f1']
FPOS  = np.zeros((len(time),1))
FVEL  = np.zeros((len(time),1))

# output expressions:
OUTP_LBL = ['global.ee_x','global.ee_y','global.ee_z']
OUTP     = np.zeros((len(time),3))


In [ ]:
from etasl_py.etasl import  etasl,to_rad,to_deg,integrate,array_to_dict,dict_to_array
import math
import numpy as np


e = etasl(nWSR=300, cputime=10, regularization_factor=1E-3)
e.readTaskSpecificationString(etaslspec)
initial_jpos = to_rad({'q1':0, 'q2':30,'q3':0,'q4':60,'q5':0,'q6':90,'q7':0})

e.initialize(initial_jpos,max_time=3,time_step=0.05,convergence_crit=1E-4)
jpos = initial_jpos

jpos["time"] = time[0] 

i =0
fpos     = {'f1':0.0}
while True:
    e.setInput(array_to_dict(INP[i,:],INP_LBL))
    e.setInputVelocity(array_to_dict(INPVEL[i,:],INP_LBL))
    
    e.setJointPos(jpos)
    e.setJointPos(fpos)
    e.solve()
    jvel = e.getJointVel(1)
    fvel = e.getJointVel(2)
    jpos = integrate(jpos, jvel, dt)
    fpos = integrate(fpos, fvel, dt)
    outp = e.getOutput()
    OUTP[i,:] = dict_to_array(outp, OUTP_LBL)

    JPOS[i,:] = dict_to_array(jpos, J_LBL)
    JVEL[i,:] = dict_to_array(jvel, J_LBL)
    FPOS[i,:] = dict_to_array(fpos, F_LBL)
    FVEL[i,:] = dict_to_array(fvel, F_LBL)
    i    = i + 1
    if i >= len(time):
        break
    #print(outp)
    

### Plot the result of the simulation:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(1,figsize=(14,4))
plt.subplot(1,2,1)
plt.plot(time,JPOS*180.0/np.pi)
plt.title('joint positions [DEG]')
plt.xlabel("time [s]")

plt.subplot(1,2,2)
plt.plot(time,JVEL)
plt.title('joint velocities [RAD/s]')
plt.xlabel("time [s]")

plt.figure(2)
plt.plot(time,OUTP,'b')
plt.plot(time,INP,'r')
plt.title('end effector and target position')
plt.xlabel("time [s]")


plt.figure(3)
plt.plot(time,FPOS)
plt.title('feature variables')
plt.xlabel("time [s]")



plt.figure(4)
plt.plot(time,OUTP-INP)
plt.title('control error')
plt.xlabel("time [s]")
#plt.ylim(-1E-3,1E-3)
plt.show()